[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/nucleo_h753zi_tensorflow_lite_micro.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/nucleo_h753zi_tensorflow_lite_micro.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/nucleo_h753zi_tensorflow_lite_micro.py)

## Install requirements

In [1]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git
! pip install -q git+https://github.com/antmicro/pyrenode3.git
! renode-run download --renode-variant dotnet-portable

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Downloaded 72.52MB / 72.52MB (time elapsed: 0:00:00.759494)...
Download finished!
Renode is already present in /root/.config/renode/renode-run.download/dotnet-portable/renode-1.15.3+20250717gitf8a0feddf


## Start Renode

In [2]:
import os
from renode_run import get_default_renode_path
from renode_run.utils import RenodeVariant

os.environ['PYRENODE_RUNTIME'] = 'coreclr'
os.environ['PYRENODE_BIN'] = get_default_renode_path(variant=RenodeVariant.DOTNET_PORTABLE)

from pyrenode3.wrappers import Emulation, Monitor, TerminalTester, LEDTester
from Antmicro.Renode.Peripherals.UART import UARTBackend
from Antmicro.Renode.Analyzers import LoggingUartAnalyzer
from System import String

currentDirectory = os.getcwd()
emulation = Emulation()
monitor = Monitor()
emulation.BackendManager.SetPreferredAnalyzer(UARTBackend, LoggingUartAnalyzer)

Renode found in /root/.config/renode/renode-run.download/dotnet-portable/renode-1.15.3+20250717gitf8a0feddf/renode


## Setup a script

In [3]:
%%writefile script.resc
logFile $ORIGIN/sex123.log True

$name?="nucleo_h753zi"
$bin?=$ORIGIN/NUCLEO_H753ZI.elf
$repl?=$ORIGIN/nucleo_h753zi.repl

using sysbus
mach create $name

machine LoadPlatformDescription $repl



showAnalyzer usart3

macro reset
"""
    sysbus LoadELF $bin
"""

runMacro $reset

Overwriting script.resc


## Run the sample

In [4]:
monitor.execute_script(currentDirectory + "/script.resc")
machine = emulation.get_mach("nucleo_h753zi")
terminalTester = TerminalTester(machine.sysbus.usart3, 100)

terminalTester.WaitFor(String('Running PerfTest on \"network\" with random inputs (16 iterations)...'), treatAsRegex=True, pauseEmulation=True)
emulation.Dispose()

## UART output

In [5]:
from renode_colab_tools import asciinema
asciinema.display_asciicast('tensorflow_lite_micro-asciinema')

FileNotFoundError: [Errno 2] No such file or directory: 'tensorflow_lite_micro-asciinema'

## Renode metrics analysis

In [ ]:
import sys
from pathlib import Path
from renode_run import get_default_renode_path
sys.path.append(str(Path(get_default_renode_path()).parent))

from renode_colab_tools import metrics
from tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('metrics.dump')


metrics.display_metrics(parser)